In [2]:
# Section 1: Initial Data Exploration and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import os

warnings.filterwarnings('ignore')

# Load all datasets
path = 'data/'
orders = pd.read_csv(path+'olist_orders_dataset.csv')
order_items = pd.read_csv(path+'olist_order_items_dataset.csv')
customers = pd.read_csv(path+'olist_customers_dataset.csv')
products = pd.read_csv(path+'olist_products_dataset.csv')
payments = pd.read_csv(path+'olist_order_payments_dataset.csv')
reviews = pd.read_csv(path+'olist_order_reviews_dataset.csv')
sellers = pd.read_csv(path+'olist_sellers_dataset.csv')
geolocation = pd.read_csv(path+'olist_geolocation_dataset.csv')

print("Dataset shapes:")
datasets = {
    'orders': orders,
    'order_items': order_items,
    'customers': customers,
    'products': products,
    'payments': payments,
    'reviews': reviews,
    'sellers': sellers,
    'geolocation': geolocation
}

for name, df in datasets.items():
    print(f"{name}: {df.shape}")

print("\nOrders dataset info:")
print(orders.info())
print("\nOrders head:")
print(orders.head())

print("\nOrder status distribution:")
print(orders['order_status'].value_counts())

print("\nDate range check:")
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
print(f"Date range: {orders['order_purchase_timestamp'].min()} to {orders['order_purchase_timestamp'].max()}")

# Check for missing values across all datasets
print("\nMissing values summary:")
for name, df in datasets.items():
    missing = df.isnull().sum().sum()
    if missing > 0:
        print(f"{name}: {missing} missing values")
        print(df.isnull().sum()[df.isnull().sum() > 0])
        print("-" * 30)

Dataset shapes:
orders: (99441, 8)
order_items: (112650, 7)
customers: (99441, 5)
products: (32951, 9)
payments: (103886, 5)
reviews: (99224, 7)
sellers: (3095, 4)
geolocation: (1000163, 5)

Orders dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
None

Orders head:
                           orde

In [3]:
# Section 2: Data Cleaning and Quality Checks

# Focus on delivered orders only for meaningful analysis
print(f"Original orders: {len(orders)}")
orders_clean = orders[orders['order_status'] == 'delivered'].copy()
print(f"Delivered orders: {len(orders_clean)}")

# Convert date columns
date_columns = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']
for col in date_columns:
    if col in orders_clean.columns:
        orders_clean[col] = pd.to_datetime(orders_clean[col])

# Remove orders with missing delivery dates since we need them for analysis
orders_clean = orders_clean.dropna(subset=['order_delivered_customer_date'])
print(f"Orders with delivery dates: {len(orders_clean)}")

# Check for duplicate orders
duplicates = orders_clean['order_id'].duplicated().sum()
print(f"Duplicate order IDs: {duplicates}")

# Filter order items to match our clean orders
order_items_clean = order_items[order_items['order_id'].isin(orders_clean['order_id'])].copy()
print(f"Order items after filtering: {len(order_items_clean)}")

# Clean payments data
payments_clean = payments[payments['order_id'].isin(orders_clean['order_id'])].copy()
print(f"Payment records after filtering: {len(payments_clean)}")

# Check payment consistency
payment_summary = payments_clean.groupby('order_id')['payment_value'].sum().reset_index()
payment_summary.columns = ['order_id', 'total_payment']

# Clean reviews data
reviews_clean = reviews[reviews['order_id'].isin(orders_clean['order_id'])].copy()
reviews_clean['review_creation_date'] = pd.to_datetime(reviews_clean['review_creation_date'])
print(f"Reviews after filtering: {len(reviews_clean)}")

# Handle missing review scores (fill with median)
median_score = reviews_clean['review_score'].median()
reviews_clean['review_score'] = reviews_clean['review_score'].fillna(median_score)

# Create product category translation dict (sample of common ones)
category_translation = {
    'beleza_saude': 'health_beauty',
    'informatica_acessorios': 'computers_accessories',
    'automotivo': 'automotive',
    'cama_mesa_banho': 'bed_bath_table',
    'moveis_decoracao': 'furniture_decor',
    'esporte_lazer': 'sports_leisure',
    'perfumaria': 'perfumery',
    'utilidades_domesticas': 'housewares',
    'telefonia': 'telephony',
    'relogios_presentes': 'watches_gifts',
    'ferramentas_jardim': 'garden_tools',
    'fashion_moda_praia': 'fashion_beach',
    'cool_stuff': 'cool_stuff',
    'brinquedos': 'toys',
    'construcao_ferramentas_construcao': 'construction_tools',
    'eletroportateis': 'small_appliances',
    'casa_construcao': 'home_construction',
    'instrumentos_musicais': 'musical_instruments',
    'eletrodomesticos': 'home_appliances',
    'livros_interesse_geral': 'books_general_interest'
}

# Apply translation where available
products['product_category_name_english'] = products['product_category_name'].map(category_translation)
products['product_category_name_english'] = products['product_category_name_english'].fillna(products['product_category_name'])

# Clean geolocation data - remove duplicates and outliers
geo_clean = geolocation.drop_duplicates(subset=['geolocation_zip_code_prefix'])

# Remove obvious outliers in coordinates (focus on Brazil)
geo_clean = geo_clean[
    (geo_clean['geolocation_lat'].between(-35, 10)) &
    (geo_clean['geolocation_lng'].between(-75, -30))
]

print(f"Geolocation records after cleaning: {len(geo_clean)}")

# Verify data integrity
print("\nData integrity checks:")
print(f"Orders with items: {orders_clean['order_id'].isin(order_items_clean['order_id']).sum()}")
print(f"Orders with payments: {orders_clean['order_id'].isin(payments_clean['order_id']).sum()}")
print(f"Orders with reviews: {orders_clean['order_id'].isin(reviews_clean['order_id']).sum()}")

print("\nCleaning completed successfully.")

Original orders: 99441
Delivered orders: 96478
Orders with delivery dates: 96470
Duplicate order IDs: 0
Order items after filtering: 110189
Payment records after filtering: 100748
Reviews after filtering: 96353
Geolocation records after cleaning: 19007

Data integrity checks:
Orders with items: 96470
Orders with payments: 96469
Orders with reviews: 95824

Cleaning completed successfully.


In [4]:
# Section 3: Feature Engineering

# Create comprehensive order dataset
print("Building master dataset...")

# Start with orders and add delivery metrics
master_df = orders_clean.copy()

# Calculate delivery performance
master_df['delivery_days'] = (master_df['order_delivered_customer_date'] - master_df['order_purchase_timestamp']).dt.days
master_df['estimated_delivery_days'] = (pd.to_datetime(master_df['order_estimated_delivery_date']) - master_df['order_purchase_timestamp']).dt.days
master_df['delivery_vs_estimate'] = master_df['delivery_days'] - master_df['estimated_delivery_days']
master_df['on_time_delivery'] = (master_df['delivery_vs_estimate'] <= 0).astype(int)

# Add temporal features
master_df['order_year'] = master_df['order_purchase_timestamp'].dt.year
master_df['order_month'] = master_df['order_purchase_timestamp'].dt.month
master_df['order_dayofweek'] = master_df['order_purchase_timestamp'].dt.dayofweek
master_df['order_quarter'] = master_df['order_purchase_timestamp'].dt.quarter

# Add customer info
master_df = master_df.merge(customers, on='customer_id', how='left')

# Add order value and item metrics
order_metrics = order_items_clean.groupby('order_id').agg({
    'price': ['sum', 'mean', 'count'],
    'freight_value': 'sum',
    'product_id': 'nunique'
}).reset_index()

order_metrics.columns = ['order_id', 'order_value', 'avg_item_price', 'item_count', 'freight_value', 'unique_products']
master_df = master_df.merge(order_metrics, on='order_id', how='left')

# Add payment info
payment_metrics = payments_clean.groupby('order_id').agg({
    'payment_value': 'sum',
    'payment_installments': 'max',
    'payment_type': lambda x: x.mode().iloc[0] if not x.empty else 'unknown'
}).reset_index()

payment_metrics.columns = ['order_id', 'total_payment', 'max_installments', 'primary_payment_type']
master_df = master_df.merge(payment_metrics, on='order_id', how='left')

# Add review data
review_metrics = reviews_clean.groupby('order_id').agg({
    'review_score': 'first',
    'review_comment_message': lambda x: (x.notna()).sum()
}).reset_index()

review_metrics.columns = ['order_id', 'review_score', 'has_review_comment']
master_df = master_df.merge(review_metrics, on='order_id', how='left')

# Fill missing review scores with median
master_df['review_score'] = master_df['review_score'].fillna(master_df['review_score'].median())

# Add product category info
order_categories = order_items_clean.merge(products[['product_id', 'product_category_name_english']], on='product_id', how='left')
primary_category = order_categories.groupby('order_id')['product_category_name_english'].first().reset_index()
primary_category.columns = ['order_id', 'primary_category']
master_df = master_df.merge(primary_category, on='order_id', how='left')

# Create customer behavior features
print("Calculating customer behavior metrics...")

# Customer lifetime metrics
customer_metrics = master_df.groupby('customer_unique_id').agg({
    'order_id': 'count',
    'order_value': ['sum', 'mean'],
    'order_purchase_timestamp': ['min', 'max'],
    'review_score': 'mean',
    'on_time_delivery': 'mean'
}).reset_index()

customer_metrics.columns = ['customer_unique_id', 'total_orders', 'total_spent', 'avg_order_value', 
                           'first_order_date', 'last_order_date', 'avg_review_score', 'on_time_delivery_rate']

# Calculate customer tenure and recency
reference_date = master_df['order_purchase_timestamp'].max()
customer_metrics['customer_tenure_days'] = (customer_metrics['last_order_date'] - customer_metrics['first_order_date']).dt.days
customer_metrics['days_since_last_order'] = (reference_date - customer_metrics['last_order_date']).dt.days

# Merge back to master dataset
master_df = master_df.merge(customer_metrics, on='customer_unique_id', how='left')

# Create RFM features
print("Creating RFM features...")
master_df['recency'] = master_df['days_since_last_order']
master_df['frequency'] = master_df['total_orders']
master_df['monetary'] = master_df['total_spent']

# Add geographic features
geo_mapping = geo_clean.groupby('geolocation_zip_code_prefix').agg({
    'geolocation_lat': 'mean',
    'geolocation_lng': 'mean',
    'geolocation_city': 'first',
    'geolocation_state': 'first'
}).reset_index()

customers_with_geo = customers.merge(geo_mapping, 
                                   left_on='customer_zip_code_prefix', 
                                   right_on='geolocation_zip_code_prefix', 
                                   how='left')

# Add state info to master dataset
state_info = customers_with_geo[['customer_id', 'geolocation_state']].rename(columns={'geolocation_state': 'customer_state'})
master_df = master_df.merge(state_info, on='customer_id', how='left')

print(f"Master dataset shape: {master_df.shape}")
print(f"Features created: {master_df.columns.tolist()}")

# Save checkpoint
master_df.to_csv('master_dataset.csv', index=False)
print("Master dataset saved to master_dataset.csv")

print("\nSample of engineered features:")
print(master_df[['order_id', 'delivery_days', 'order_value', 'review_score', 'recency', 'frequency', 'monetary']].head())

Building master dataset...
Calculating customer behavior metrics...
Creating RFM features...
Master dataset shape: (96470, 44)
Features created: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'delivery_days', 'estimated_delivery_days', 'delivery_vs_estimate', 'on_time_delivery', 'order_year', 'order_month', 'order_dayofweek', 'order_quarter', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state_x', 'order_value', 'avg_item_price', 'item_count', 'freight_value', 'unique_products', 'total_payment', 'max_installments', 'primary_payment_type', 'review_score', 'has_review_comment', 'primary_category', 'total_orders', 'total_spent', 'avg_order_value', 'first_order_date', 'last_order_date', 'avg_review_score', 'on_time_delivery_rate', 'customer_tenure_days', 'days_since_last_order', 'recency', 'frequency', 'monetary', 'cu

In [7]:
# Section 4: Customer Segmentation using RFM Analysis

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

print("Starting RFM Analysis...")

# First, let's check what columns we have
print("Available columns in master_df:")
print([col for col in master_df.columns if any(keyword in col.lower() for keyword in ['customer', 'days', 'total', 'avg', 'state'])])

# Create customer-level RFM dataset with available columns
agg_dict = {
    'days_since_last_order': 'first',  # Recency
    'total_orders': 'first',           # Frequency  
    'total_spent': 'first',            # Monetary
    'avg_review_score': 'first'
}

# Add state column if it exists
state_columns = [col for col in master_df.columns if 'state' in col.lower()]
if state_columns:
    agg_dict[state_columns[0]] = 'first'
    print(f"Using state column: {state_columns[0]}")
else:
    print("No state column found, continuing without geographic data")

rfm_data = master_df.groupby('customer_unique_id').agg(agg_dict).reset_index()

# Set column names dynamically
base_columns = ['customer_id', 'recency', 'frequency', 'monetary', 'avg_review_score']
if state_columns:
    rfm_data.columns = base_columns + ['customer_state']
else:
    rfm_data.columns = base_columns

# Remove customers with missing RFM values
rfm_data = rfm_data.dropna(subset=['recency', 'frequency', 'monetary'])
print(f"Customers for RFM analysis: {len(rfm_data)}")

# Log transform monetary values to handle skewness
rfm_data['monetary_log'] = np.log1p(rfm_data['monetary'])

# Create RFM scores using quantiles
rfm_data['R_score'] = pd.qcut(rfm_data['recency'], 5, labels=[5,4,3,2,1])  # Lower recency = higher score
rfm_data['F_score'] = pd.qcut(rfm_data['frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm_data['M_score'] = pd.qcut(rfm_data['monetary'], 5, labels=[1,2,3,4,5])

# Convert to numeric
rfm_data['R_score'] = pd.to_numeric(rfm_data['R_score'])
rfm_data['F_score'] = pd.to_numeric(rfm_data['F_score'])
rfm_data['M_score'] = pd.to_numeric(rfm_data['M_score'])

# Create combined RFM score
rfm_data['RFM_score'] = rfm_data['R_score'].astype(str) + rfm_data['F_score'].astype(str) + rfm_data['M_score'].astype(str)

# Traditional RFM segments
def categorize_rfm(row):
    if row['RFM_score'] in ['555', '554', '544', '545', '454', '455', '445']:
        return 'Champions'
    elif row['RFM_score'] in ['543', '444', '435', '355', '354', '345', '344', '335']:
        return 'Loyal Customers'
    elif row['RFM_score'] in ['553', '551', '552', '541', '542', '533', '532', '531', '452', '451']:
        return 'Potential Loyalists'
    elif row['RFM_score'] in ['512', '511', '422', '421', '412', '411', '311']:
        return 'New Customers'
    elif row['RFM_score'] in ['155', '154', '144', '214', '215', '115', '114']:
        return 'At Risk'
    elif row['RFM_score'] in ['155', '154', '144', '214', '215', '115']:
        return 'Cannot Lose Them'
    elif row['RFM_score'] in ['331', '321', '231', '241', '251']:
        return 'About to Sleep'
    elif row['RFM_score'] in ['111', '112', '121', '131', '141', '151']:
        return 'Lost'
    else:
        return 'Others'

rfm_data['rfm_segment'] = rfm_data.apply(categorize_rfm, axis=1)

# K-means clustering for validation
print("Performing K-means clustering...")

# Prepare data for clustering
rfm_features = rfm_data[['recency', 'frequency', 'monetary_log']].copy()
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_features)

# Find optimal number of clusters
silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(rfm_scaled)
    silhouette_avg = silhouette_score(rfm_scaled, cluster_labels)
    silhouette_scores.append(silhouette_avg)

optimal_k = k_range[np.argmax(silhouette_scores)]
print(f"Optimal number of clusters: {optimal_k}")

# Apply optimal clustering
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
rfm_data['cluster'] = kmeans_final.fit_predict(rfm_scaled)

# Analyze clusters
print("\nCluster Analysis:")
cluster_summary = rfm_data.groupby('cluster').agg({
    'recency': ['mean', 'count'],
    'frequency': 'mean',
    'monetary': 'mean',
    'avg_review_score': 'mean'
}).round(2)

cluster_summary.columns = ['avg_recency', 'customer_count', 'avg_frequency', 'avg_monetary', 'avg_review_score']
print(cluster_summary)

# Create cluster labels based on characteristics
def label_cluster(cluster_num):
    cluster_data = rfm_data[rfm_data['cluster'] == cluster_num]
    avg_recency = cluster_data['recency'].mean()
    avg_frequency = cluster_data['frequency'].mean()
    avg_monetary = cluster_data['monetary'].mean()
    
    if avg_recency < 100 and avg_frequency > 2 and avg_monetary > 300:
        return 'High Value Active'
    elif avg_recency < 100 and avg_frequency <= 2:
        return 'Recent Customers'
    elif avg_frequency > 2 and avg_monetary > 200:
        return 'Loyal Customers'
    elif avg_recency > 200:
        return 'At Risk/Lost'
    else:
        return 'Standard Customers'

cluster_labels = {i: label_cluster(i) for i in range(optimal_k)}
rfm_data['cluster_label'] = rfm_data['cluster'].map(cluster_labels)

print("\nCluster Labels:")
for cluster_id, label in cluster_labels.items():
    count = (rfm_data['cluster'] == cluster_id).sum()
    print(f"Cluster {cluster_id}: {label} ({count} customers)")

# Traditional RFM segment analysis
print("\nTraditional RFM Segments:")
segment_summary = rfm_data.groupby('rfm_segment').agg({
    'customer_id': 'count',
    'recency': 'mean',
    'frequency': 'mean', 
    'monetary': 'mean'
}).round(2)

segment_summary.columns = ['customer_count', 'avg_recency', 'avg_frequency', 'avg_monetary']
segment_summary = segment_summary.sort_values('customer_count', ascending=False)
print(segment_summary)

# Geographic distribution by segment
print("\nTop states by segment:")
geo_segment = rfm_data.groupby(['customer_state', 'cluster_label']).size().reset_index(name='count')
for label in rfm_data['cluster_label'].unique():
    if pd.notna(label):
        top_states = geo_segment[geo_segment['cluster_label'] == label].nlargest(3, 'count')
        print(f"\n{label}:")
        for _, row in top_states.iterrows():
            if pd.notna(row['customer_state']):
                print(f"  {row['customer_state']}: {row['count']} customers")

# Export results
customers_segmented = rfm_data[['customer_id', 'recency', 'frequency', 'monetary', 
                               'rfm_segment', 'cluster', 'cluster_label', 'customer_state']].copy()

customers_segmented.to_csv('customers_segmented.csv', index=False)
print(f"\nCustomer segmentation completed. Results saved to customers_segmented.csv")
print(f"Total customers segmented: {len(customers_segmented)}")

# Quick validation
print("\nSegmentation Summary:")
print(customers_segmented['cluster_label'].value_counts())

Starting RFM Analysis...
Available columns in master_df:
['customer_id', 'order_delivered_customer_date', 'delivery_days', 'estimated_delivery_days', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state_x', 'avg_item_price', 'total_payment', 'total_orders', 'total_spent', 'avg_order_value', 'avg_review_score', 'customer_tenure_days', 'days_since_last_order', 'customer_state_y']
Using state column: customer_state_x
Customers for RFM analysis: 93350
Performing K-means clustering...
Optimal number of clusters: 2

Cluster Analysis:
         avg_recency  customer_count  avg_frequency  avg_monetary  \
cluster                                                             
0             237.50           90549           1.00        137.96   
1             219.29            2801           2.11        260.05   

         avg_review_score  
cluster                    
0                    4.16  
1                    4.22  

Cluster Labels:
Cluster 0: At Risk/Lost (90549

In [14]:
# Section 5: Churn Prediction Modeling

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

print("Building churn prediction model...")

# Define churn based on recency (no orders in last 180 days)
CHURN_THRESHOLD = 180

# Create churn dataset from customer-level data
churn_data = master_df.groupby('customer_unique_id').agg({
    'days_since_last_order': 'first',
    'total_orders': 'first',
    'total_spent': 'first',
    'avg_order_value': 'first',
    'avg_review_score': 'first',
    'on_time_delivery_rate': 'first',
    'customer_tenure_days': 'first',
    'customer_state_y': 'first',  # Using the correct column name
    'primary_category': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'unknown'
}).reset_index()

churn_data.columns = ['customer_id', 'days_since_last_order', 'total_orders', 'total_spent', 
                     'avg_order_value', 'avg_review_score', 'on_time_delivery_rate', 
                     'customer_tenure_days', 'customer_state', 'primary_category']

# Define churn target
churn_data['is_churned'] = (churn_data['days_since_last_order'] > CHURN_THRESHOLD).astype(int)

print(f"Total customers: {len(churn_data)}")
print(f"Churned customers: {churn_data['is_churned'].sum()} ({churn_data['is_churned'].mean():.2%})")

# Feature engineering for churn model
churn_data['order_frequency'] = churn_data['total_orders'] / (churn_data['customer_tenure_days'] + 1) * 30  # orders per month
churn_data['clv_estimate'] = churn_data['avg_order_value'] * churn_data['order_frequency'] * 12  # annual CLV estimate

# Handle missing values
numeric_columns = ['total_orders', 'total_spent', 'avg_order_value', 'avg_review_score', 
                  'on_time_delivery_rate', 'customer_tenure_days', 'order_frequency', 'clv_estimate']

for col in numeric_columns:
    churn_data[col] = churn_data[col].fillna(churn_data[col].median())

# Fill categorical missing values
churn_data['customer_state'] = churn_data['customer_state'].fillna('Unknown')
churn_data['primary_category'] = churn_data['primary_category'].fillna('unknown')

# Encode categorical variables
le_state = LabelEncoder()
le_category = LabelEncoder()

churn_data['state_encoded'] = le_state.fit_transform(churn_data['customer_state'])
churn_data['category_encoded'] = le_category.fit_transform(churn_data['primary_category'])

# Select features for modeling
feature_columns = ['total_orders', 'total_spent', 'avg_order_value', 'avg_review_score', 
                  'on_time_delivery_rate', 'customer_tenure_days', 'order_frequency', 
                  'clv_estimate', 'state_encoded', 'category_encoded']

# Remove customers with insufficient history (less than 30 days tenure)
model_data = churn_data[churn_data['customer_tenure_days'] >= 30].copy()
print(f"Customers with sufficient history: {len(model_data)}")

X = model_data[feature_columns]
y = model_data['is_churned']

# Check for any remaining missing values
print(f"Missing values in features: {X.isnull().sum().sum()}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"Training churn rate: {y_train.mean():.2%}")

# Model comparison
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42, n_estimators=100)
}

model_results = {}

print("\nModel Comparison (Cross-Validation):")
for name, model in models.items():
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
    model_results[name] = {
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'model': model
    }
    print(f"{name}: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Select best model
best_model_name = max(model_results.keys(), key=lambda k: model_results[k]['cv_mean'])
best_model = model_results[best_model_name]['model']

print(f"\nBest model: {best_model_name}")

# Hyperparameter tuning for best model
if best_model_name == 'Random Forest':
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }
elif best_model_name == 'Gradient Boosting':
    param_grid = {
        'n_estimators': [100, 200],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [3, 5, 7]
    }
else:  # Logistic Regression
    param_grid = {
        'C': [0.1, 1.0, 10.0],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga']
    }

print("Performing hyperparameter tuning...")
grid_search = GridSearchCV(best_model, param_grid, cv=3, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train, y_train)

final_model = grid_search.best_estimator_
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV score: {grid_search.best_score_:.4f}")

# Train final model and make predictions
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
y_pred_proba = final_model.predict_proba(X_test)[:, 1]

# Evaluate model
print("\nModel Performance:")
print(f"Test AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Feature importance analysis
if hasattr(final_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': feature_columns,
        'importance': final_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nTop 10 Feature Importances:")
    print(feature_importance.head(10))
    
    # Plot feature importance
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(feature_importance)), feature_importance['importance'])
    plt.yticks(range(len(feature_importance)), feature_importance['feature'])
    plt.xlabel('Feature Importance')
    plt.title('Feature Importance for Churn Prediction')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

# Create predictions for all customers
all_predictions = final_model.predict_proba(X)[:, 1]
model_data['churn_probability'] = all_predictions
model_data['churn_risk_category'] = pd.cut(all_predictions, 
                                          bins=[0, 0.3, 0.7, 1.0], 
                                          labels=['Low Risk', 'Medium Risk', 'High Risk'])

print("\nChurn Risk Distribution:")
print(model_data['churn_risk_category'].value_counts())

# Business insights
high_risk_customers = model_data[model_data['churn_risk_category'] == 'High Risk']
print(f"\nHigh-risk customers: {len(high_risk_customers)}")
print(f"Potential revenue at risk: ${high_risk_customers['clv_estimate'].sum():,.2f}")

# Export churn predictions
churn_predictions = model_data[['customer_id', 'is_churned', 'churn_probability', 
                               'churn_risk_category', 'total_spent', 'clv_estimate',
                               'customer_state', 'primary_category']].copy()

churn_predictions.to_csv('churn_predictions.csv', index=False)
print(f"\nChurn predictions saved to churn_predictions.csv")
print(f"Total customers analyzed: {len(churn_predictions)}")

# Model insights summary
print("\nKey Insights:")
print(f"1. Model can identify churned customers with {roc_auc_score(y_test, y_pred_proba):.1%} accuracy (AUC)")
print(f"2. {len(high_risk_customers)} customers are at high risk of churning")
print(f"3. Potential revenue at risk: ${high_risk_customers['clv_estimate'].sum():,.0f}")

if hasattr(final_model, 'feature_importances_'):
    top_feature = feature_importance.iloc[0]['feature']
    print(f"4. Most important predictor: {top_feature}")

print("Churn modeling completed successfully.")

Building churn prediction model...
Total customers: 93350
Churned customers: 55004 (58.92%)
Customers with sufficient history: 1457
Missing values in features: 0
Training set: 1165 samples
Test set: 292 samples
Training churn rate: 42.49%

Model Comparison (Cross-Validation):
Logistic Regression: 0.6349 (+/- 0.0936)
Random Forest: 0.6056 (+/- 0.0681)
Gradient Boosting: 0.6144 (+/- 0.0659)

Best model: Logistic Regression
Performing hyperparameter tuning...
Best parameters: {'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}
Best CV score: 0.6431

Model Performance:
Test AUC: 0.6225

Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.74      0.67       168
           1       0.50      0.35      0.42       124

    accuracy                           0.58       292
   macro avg       0.55      0.55      0.54       292
weighted avg       0.56      0.58      0.56       292


Confusion Matrix:
[[124  44]
 [ 80  44]]

Churn Risk Distribution

In [19]:
# Section 6: Product and Sales Analysis

print("Analyzing product performance and sales patterns...")

# Create comprehensive product analysis dataset
product_analysis = order_items_clean.merge(products[['product_id', 'product_category_name_english']], 
                                         on='product_id', how='left')
product_analysis = product_analysis.merge(orders_clean[['order_id', 'customer_id', 'order_purchase_timestamp']], 
                                        on='order_id', how='left')
# Add customer state by merging with customers table
product_analysis = product_analysis.merge(customers[['customer_id', 'customer_state']], 
                                        on='customer_id', how='left')
product_analysis = product_analysis.merge(reviews_clean[['order_id', 'review_score']], 
                                        on='order_id', how='left')

# Add temporal features
product_analysis['order_year'] = pd.to_datetime(product_analysis['order_purchase_timestamp']).dt.year
product_analysis['order_month'] = pd.to_datetime(product_analysis['order_purchase_timestamp']).dt.month
product_analysis['order_quarter'] = pd.to_datetime(product_analysis['order_purchase_timestamp']).dt.quarter

# Category performance analysis
print("Analyzing category performance...")
category_performance = product_analysis.groupby('product_category_name_english').agg({
    'price': ['sum', 'mean', 'count'],
    'freight_value': ['sum', 'mean'],
    'review_score': 'mean',
    'product_id': 'nunique',
    'order_id': 'nunique'
}).round(2)

category_performance.columns = ['total_revenue', 'avg_price', 'items_sold', 'total_freight', 
                               'avg_freight', 'avg_review_score', 'unique_products', 'unique_orders']

# Calculate profitability metrics (assuming freight is a cost)
category_performance['gross_revenue'] = category_performance['total_revenue']
category_performance['estimated_profit'] = category_performance['total_revenue'] - category_performance['total_freight']
category_performance['profit_margin'] = (category_performance['estimated_profit'] / category_performance['total_revenue'] * 100).round(2)

# Add market share
total_revenue = category_performance['total_revenue'].sum()
category_performance['market_share'] = (category_performance['total_revenue'] / total_revenue * 100).round(2)

category_performance = category_performance.sort_values('total_revenue', ascending=False)

print("Top 10 Categories by Revenue:")
print(category_performance.head(10)[['total_revenue', 'items_sold', 'avg_review_score', 'market_share']])

# Seasonal analysis
print("\nAnalyzing seasonal patterns...")
monthly_sales = product_analysis.groupby(['order_year', 'order_month']).agg({
    'price': 'sum',
    'order_id': 'nunique'
}).reset_index()

monthly_sales['revenue'] = monthly_sales['price']
monthly_sales['orders'] = monthly_sales['order_id']
monthly_sales['date'] = pd.to_datetime(monthly_sales[['order_year', 'order_month']].rename(columns={'order_year': 'year', 'order_month': 'month'}).assign(day=1))

# Calculate month-over-month growth
monthly_sales = monthly_sales.sort_values('date')
monthly_sales['revenue_growth'] = monthly_sales['revenue'].pct_change() * 100
monthly_sales['orders_growth'] = monthly_sales['orders'].pct_change() * 100

print("Monthly sales trends (last 6 months):")
print(monthly_sales.tail(6)[['order_year', 'order_month', 'revenue', 'orders', 'revenue_growth']])

# Seasonal category analysis
seasonal_categories = product_analysis.groupby(['product_category_name_english', 'order_quarter']).agg({
    'price': 'sum',
    'order_id': 'count'
}).reset_index()

# Find categories with highest seasonal variation
category_seasonal_variation = seasonal_categories.groupby('product_category_name_english')['price'].std().sort_values(ascending=False)

print("\nCategories with highest seasonal variation:")
print(category_seasonal_variation.head(10))

# Geographic analysis
print("\nAnalyzing geographic performance...")
geographic_performance = product_analysis.groupby('customer_state').agg({
    'price': ['sum', 'count'],
    'review_score': 'mean',
    'order_id': 'nunique'
}).round(2)

geographic_performance.columns = ['total_revenue', 'items_sold', 'avg_review_score', 'unique_orders']
geographic_performance['avg_order_value'] = (geographic_performance['total_revenue'] / geographic_performance['unique_orders']).round(2)

# Add market share by state
total_revenue = geographic_performance['total_revenue'].sum()
geographic_performance['market_share'] = (geographic_performance['total_revenue'] / total_revenue * 100).round(2)

geographic_performance = geographic_performance.sort_values('total_revenue', ascending=False)

print("Top 10 States by Revenue:")
print(geographic_performance.head(10)[['total_revenue', 'unique_orders', 'avg_order_value', 'market_share']])

# Product performance segmentation
print("\nSegmenting products by performance...")

# Calculate product-level metrics
product_metrics = product_analysis.groupby('product_id').agg({
    'price': ['sum', 'count'],
    'review_score': 'mean',
    'order_id': 'nunique'
}).round(2)

product_metrics.columns = ['total_revenue', 'units_sold', 'avg_review_score', 'unique_orders']
product_metrics = product_metrics.reset_index()

# Create performance segments
revenue_quartiles = product_metrics['total_revenue'].quantile([0.25, 0.5, 0.75, 1.0])
units_quartiles = product_metrics['units_sold'].quantile([0.25, 0.5, 0.75, 1.0])

def categorize_product(row):
    revenue = row['total_revenue']
    units = row['units_sold']
    
    high_revenue = revenue >= revenue_quartiles[0.75]
    high_volume = units >= units_quartiles[0.75]
    low_revenue = revenue <= revenue_quartiles[0.25]
    low_volume = units <= units_quartiles[0.25]
    
    if high_revenue and high_volume:
        return 'Star Products'
    elif high_revenue and not high_volume:
        return 'Premium Products' 
    elif not high_revenue and high_volume:
        return 'Volume Products'
    elif low_revenue and low_volume:
        return 'Underperformers'
    else:
        return 'Standard Products'

product_metrics['performance_segment'] = product_metrics.apply(categorize_product, axis=1)

print("Product Performance Segments:")
segment_summary = product_metrics.groupby('performance_segment').agg({
    'product_id': 'count',
    'total_revenue': 'sum',
    'avg_review_score': 'mean'
}).round(2)

segment_summary.columns = ['product_count', 'total_revenue', 'avg_review_score']
print(segment_summary)

# Export comprehensive results
print("\nExporting analysis results...")

# Category performance export
category_performance_export = category_performance.reset_index()
category_performance_export.to_csv('product_performance.csv', index=False)

# Monthly trends export
monthly_sales.to_csv('seasonal_trends.csv', index=False)

# Geographic analysis export
geographic_performance_export = geographic_performance.reset_index()
geographic_performance_export.to_csv('geographic_analysis.csv', index=False)

# Combined insights dataset
insights_summary = {
    'top_category': category_performance.index[0],
    'top_category_revenue': category_performance.iloc[0]['total_revenue'],
    'total_categories': len(category_performance),
    'avg_review_score_overall': product_analysis['review_score'].mean(),
    'top_state': geographic_performance.index[0],
    'top_state_revenue': geographic_performance.iloc[0]['total_revenue'],
    'seasonal_peak_month': monthly_sales.loc[monthly_sales['revenue'].idxmax(), 'order_month'],
    'total_products': len(product_metrics),
    'star_products_count': (product_metrics['performance_segment'] == 'Star Products').sum()
}

insights_df = pd.DataFrame([insights_summary])
insights_df.to_csv('business_insights.csv', index=False)

print(f"Analysis completed successfully!")
print(f"Files exported:")
print(f"- product_performance.csv ({len(category_performance_export)} categories)")
print(f"- seasonal_trends.csv ({len(monthly_sales)} months)")
print(f"- geographic_analysis.csv ({len(geographic_performance_export)} states)")
print(f"- business_insights.csv (key metrics)")

print(f"\nKey Business Insights:")
print(f"1. Top performing category: {insights_summary['top_category']} (${insights_summary['top_category_revenue']:,.0f})")
print(f"2. Top performing state: {insights_summary['top_state']} (${insights_summary['top_state_revenue']:,.0f})")
print(f"3. Peak sales month: {insights_summary['seasonal_peak_month']}")
print(f"4. Star products identified: {insights_summary['star_products_count']}")
print(f"5. Overall customer satisfaction: {insights_summary['avg_review_score_overall']:.2f}/5.0")

Analyzing product performance and sales patterns...
Analyzing category performance...
Top 10 Categories by Revenue:
                               total_revenue  items_sold  avg_review_score  \
product_category_name_english                                                
health_beauty                     1237439.95        9519              4.19   
watches_gifts                     1166968.63        5867              4.07   
bed_bath_table                    1037177.69       11107              3.92   
sports_leisure                     960010.09        8488              4.17   
computers_accessories              896132.29        7707              3.99   
furniture_decor                    718344.78        8239              3.95   
housewares                         617836.73        6819              4.11   
cool_stuff                         612071.86        3727              4.19   
automotive                         580146.14        4157              4.12   
toys                      

In [20]:
# Section 7: Final Data Integration and Tableau Export Preparation

print("Preparing final datasets for Tableau integration...")

# Create master customer dataset with all insights
print("Building comprehensive customer dataset...")

# Load our previously created datasets
customers_segmented = pd.read_csv('customers_segmented.csv')
churn_predictions = pd.read_csv('churn_predictions.csv')

# Merge customer insights
customer_master = customers_segmented.merge(
    churn_predictions[['customer_id', 'churn_probability', 'churn_risk_category', 'clv_estimate']], 
    on='customer_id', 
    how='left'
)

# Add geographic revenue data
customer_revenue_by_state = geographic_performance_export.reset_index()
customer_master = customer_master.merge(
    customer_revenue_by_state[['customer_state', 'total_revenue', 'market_share']].rename(columns={
        'total_revenue': 'state_total_revenue',
        'market_share': 'state_market_share'
    }),
    on='customer_state',
    how='left'
)

print(f"Customer master dataset: {customer_master.shape}")

# Create order-level dataset for detailed analysis
print("Building comprehensive order dataset...")

# Start with our clean master dataset
order_master = master_df.copy()

# Add customer segment information
order_master = order_master.merge(
    customers_segmented[['customer_id', 'cluster_label', 'rfm_segment']], 
    on='customer_id', 
    how='left'
)

# Add churn risk information
order_master = order_master.merge(
    churn_predictions[['customer_id', 'churn_probability', 'churn_risk_category']], 
    left_on='customer_unique_id',
    right_on='customer_id',
    how='left',
    suffixes=('', '_churn')
)

# Clean up duplicate customer_id column
if 'customer_id_churn' in order_master.columns:
    order_master = order_master.drop('customer_id_churn', axis=1)

print(f"Order master dataset: {order_master.shape}")

# Create time series dataset for trend analysis
print("Building time series dataset...")

# Daily aggregations
daily_metrics = order_master.groupby(order_master['order_purchase_timestamp'].dt.date).agg({
    'order_id': 'count',
    'order_value': ['sum', 'mean'],
    'review_score': 'mean',
    'on_time_delivery': 'mean',
    'customer_unique_id': 'nunique'
}).reset_index()

daily_metrics.columns = ['date', 'daily_orders', 'daily_revenue', 'avg_order_value', 
                        'avg_review_score', 'on_time_delivery_rate', 'unique_customers']

# Add temporal features
daily_metrics['date'] = pd.to_datetime(daily_metrics['date'])
daily_metrics['year'] = daily_metrics['date'].dt.year
daily_metrics['month'] = daily_metrics['date'].dt.month
daily_metrics['quarter'] = daily_metrics['date'].dt.quarter
daily_metrics['day_of_week'] = daily_metrics['date'].dt.day_name()
daily_metrics['is_weekend'] = daily_metrics['date'].dt.weekday >= 5

# Calculate rolling averages
daily_metrics = daily_metrics.sort_values('date')
daily_metrics['revenue_7d_avg'] = daily_metrics['daily_revenue'].rolling(7).mean()
daily_metrics['revenue_30d_avg'] = daily_metrics['daily_revenue'].rolling(30).mean()

print(f"Daily metrics dataset: {daily_metrics.shape}")

# Create category performance dataset with trends
print("Building category trends dataset...")

category_trends = product_analysis.groupby(['product_category_name_english', 'order_year', 'order_month']).agg({
    'price': 'sum',
    'order_id': 'count',
    'review_score': 'mean'
}).reset_index()

category_trends.columns = ['category', 'year', 'month', 'revenue', 'orders', 'avg_review_score']
category_trends['date'] = pd.to_datetime(category_trends[['year', 'month']].assign(day=1))

# Calculate growth rates by category
category_trends = category_trends.sort_values(['category', 'date'])
category_trends['revenue_growth'] = category_trends.groupby('category')['revenue'].pct_change() * 100
category_trends['orders_growth'] = category_trends.groupby('category')['orders'].pct_change() * 100

print(f"Category trends dataset: {category_trends.shape}")

# Create final business summary dataset
print("Creating executive summary dataset...")

# Current date reference
latest_date = order_master['order_purchase_timestamp'].max()
current_month = latest_date.month
current_year = latest_date.year

# Executive KPIs
executive_summary = {
    'total_customers': order_master['customer_unique_id'].nunique(),
    'total_orders': len(order_master),
    'total_revenue': order_master['order_value'].sum(),
    'avg_order_value': order_master['order_value'].mean(),
    'avg_review_score': order_master['review_score'].mean(),
    'on_time_delivery_rate': order_master['on_time_delivery'].mean(),
    'customers_at_risk': len(churn_predictions[churn_predictions['churn_risk_category'] == 'High Risk']),
    'revenue_at_risk': churn_predictions[churn_predictions['churn_risk_category'] == 'High Risk']['clv_estimate'].sum(),
    'top_customer_segment': customer_master['cluster_label'].mode().iloc[0],
    'top_category': category_performance_export.iloc[0]['product_category_name_english'],
    'top_state': geographic_performance_export.iloc[0]['customer_state'],
    'analysis_date': latest_date.strftime('%Y-%m-%d')
}

executive_df = pd.DataFrame([executive_summary])

print("Executive Summary:")
for key, value in executive_summary.items():
    if isinstance(value, (int, float)) and 'rate' in key:
        print(f"{key}: {value:.2%}")
    elif isinstance(value, (int, float)) and 'revenue' in key or 'value' in key:
        print(f"{key}: ${value:,.2f}")
    elif isinstance(value, (int, float)):
        print(f"{key}: {value:,.0f}")
    else:
        print(f"{key}: {value}")

# Export all datasets for Tableau
print("\nExporting final datasets...")

datasets_to_export = {
    'customer_master_tableau.csv': customer_master,
    'order_master_tableau.csv': order_master,
    'daily_metrics_tableau.csv': daily_metrics,
    'category_trends_tableau.csv': category_trends,
    'executive_summary_tableau.csv': executive_df
}

for filename, dataset in datasets_to_export.items():
    # Clean column names for Tableau compatibility
    dataset.columns = [col.replace(' ', '_').replace('(', '').replace(')', '').lower() 
                      for col in dataset.columns]
    
    # Handle datetime columns
    for col in dataset.columns:
        if dataset[col].dtype == 'datetime64[ns]':
            dataset[col] = dataset[col].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    dataset.to_csv(filename, index=False)
    print(f"✓ {filename}: {dataset.shape[0]} rows, {dataset.shape[1]} columns")

# Create data dictionary for Tableau users
print("\nCreating data dictionary...")

data_dictionary = {
    'Dataset': [],
    'Column': [],
    'Description': [],
    'Data_Type': []
}

# Customer master dictionary
customer_columns = {
    'customer_id': 'Unique customer identifier',
    'recency': 'Days since last purchase',
    'frequency': 'Total number of orders',
    'monetary': 'Total customer spend',
    'rfm_segment': 'Traditional RFM customer segment',
    'cluster_label': 'ML-based customer segment',
    'churn_probability': 'Probability of customer churn (0-1)',
    'churn_risk_category': 'Churn risk level (Low/Medium/High)',
    'clv_estimate': 'Estimated customer lifetime value'
}

for col, desc in customer_columns.items():
    data_dictionary['Dataset'].append('customer_master_tableau.csv')
    data_dictionary['Column'].append(col)
    data_dictionary['Description'].append(desc)
    data_dictionary['Data_Type'].append('String' if col in ['rfm_segment', 'cluster_label', 'churn_risk_category'] else 'Number')

# Order master key columns
order_columns = {
    'order_id': 'Unique order identifier',
    'order_value': 'Total order value in BRL',
    'review_score': 'Customer review score (1-5)',
    'delivery_days': 'Actual delivery time in days',
    'on_time_delivery': 'Whether delivered on time (1/0)',
    'cluster_label': 'Customer segment',
    'churn_probability': 'Customer churn probability'
}

for col, desc in order_columns.items():
    data_dictionary['Dataset'].append('order_master_tableau.csv')
    data_dictionary['Column'].append(col)
    data_dictionary['Description'].append(desc)
    data_dictionary['Data_Type'].append('String' if col == 'cluster_label' else 'Number')

data_dict_df = pd.DataFrame(data_dictionary)
data_dict_df.to_csv('tableau_data_dictionary.csv', index=False)

print("✓ tableau_data_dictionary.csv: Data definitions for Tableau users")

# Final validation
print("\nData validation summary:")
print(f"Customer master: {len(customer_master)} customers")
print(f"Order master: {len(order_master)} orders") 
print(f"Daily metrics: {len(daily_metrics)} days of data")
print(f"Category trends: {len(category_trends)} category-month combinations")

# Check for missing values in key columns
key_columns_check = {
    'customer_master': ['customer_id', 'cluster_label', 'churn_probability'],
    'order_master': ['order_id', 'order_value', 'review_score']
}

for dataset_name, columns in key_columns_check.items():
    dataset = customer_master if dataset_name == 'customer_master' else order_master
    missing_summary = dataset[columns].isnull().sum()
    if missing_summary.sum() > 0:
        print(f"Warning: Missing values in {dataset_name}:")
        print(missing_summary[missing_summary > 0])
    else:
        print(f"✓ {dataset_name}: No missing values in key columns")

print("\n" + "="*50)
print("DATA PREPARATION COMPLETED SUCCESSFULLY!")
print("="*50)
print("\nNext steps:")
print("1. Import the CSV files into Tableau")
print("2. Create relationships between datasets using customer_id and order_id")
print("3. Build the four recommended dashboards:")
print("   - Executive Dashboard")
print("   - Customer Analytics Dashboard") 
print("   - Product Performance Dashboard")
print("   - Predictive Insights Dashboard")
print("\nAll files are ready for professional Tableau visualization!")

Preparing final datasets for Tableau integration...
Building comprehensive customer dataset...
Customer master dataset: (93350, 13)
Building comprehensive order dataset...
Order master dataset: (96470, 48)
Building time series dataset...
Daily metrics dataset: (612, 14)
Building category trends dataset...
Category trends dataset: (1252, 9)
Creating executive summary dataset...
Executive Summary:
total_customers: 93,350
total_orders: 96,470
total_revenue: $13,220,248.93
avg_order_value: $137.04
avg_review_score: 4
on_time_delivery_rate: 92.43%
customers_at_risk: 7
revenue_at_risk: $76,353.53
top_customer_segment: At Risk/Lost
top_category: health_beauty
top_state: SP
analysis_date: 2018-08-29

Exporting final datasets...
✓ customer_master_tableau.csv: 93350 rows, 13 columns
✓ order_master_tableau.csv: 96470 rows, 48 columns
✓ daily_metrics_tableau.csv: 612 rows, 14 columns
✓ category_trends_tableau.csv: 1252 rows, 9 columns
✓ executive_summary_tableau.csv: 1 rows, 12 columns

Creating d